In [2]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

import spacy

Using TensorFlow backend.


In [7]:
import re

In [3]:
tweetdata = pd.read_csv("../data/testdata.manual.2009.06.14.csv", header=None, encoding="ISO-8859-1")
tweetdata.columns = ['polarity', 'id', 'date', 'source', 'username', 'text']

In [4]:
labels = tweetdata.polarity

In [8]:
clean_str_sst(tweetdata.text[0])

'stellargirl i loooooooovvvvvveee my kindle2 not that the dx is cool, but the 2 is fantastic in its own right'

In [11]:
clean_str(tweetdata.text[0])

'stellargirl i loooooooovvvvvveee my kindle2 not that the dx is cool , but the 2 is fantastic in its own right'

In [19]:
clean_str_sst(tweetdata.text[4])

"mikefish fair enough but i have the kindle2 and i think it's perfect )"

In [28]:
clean_str_sst('@adfaf https:ad;ajadfadf.12324234ca?')

'adfaf https ad ajadfadf 12324234ca?'

In [16]:
tweetdata.text[4]

"@mikefish  Fair enough. But i have the Kindle2 and I think it's perfect  :)"

In [25]:
clean_str_sst(tweetdata.text[4])

"mikefish fair enough but i have the kindle2 and i think it's perfect"

In [9]:
def clean_str(string):
    # 2/2/19 Adapted from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    Tokenization/string cleaning for all datasets.
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [23]:
def clean_str_sst(string):
    """
    Tokenization/string cleaning for the SST dataset
    """
    string = re.sub(r"[^A-Za-z0-9,!?\'\`]", " ", string)   
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [5]:
# 1/21/19 https://github.com/hundredblocks/concrete_NLP_tutorial/blob/master/NLP_notebook.ipynb
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

In [6]:
tweetdata = standardize_text(tweetdata, 'text')
tweet_doc = tweetdata.text.tolist()

In [15]:
import requests
import json

In [70]:
matrix_embedding = np.zeros((len(tweetdata), 300))
for i in range(len(tweetdata)):
    text = tweetdata['text'][i]
    input = {'text' : text}
    response = requests.get('http://0.0.0.0:5000/infer', data=input)
    vector_embedding = json.loads(response.text)
    matrix_embedding[i] = vector_embedding

In [72]:
matrix_embedding.shape

(498, 300)

In [21]:
# t = Tokenizer()
# t.fit_on_texts(tweet_doc)

# vocab_size = len(t.word_index) + 1
# encoded_docs = t.texts_to_sequences(tweet_doc)

In [23]:
# sentence_lengths = []
# for doc in encoded_docs:
#     sentence_lengths.append(len(doc))
# max_length = max(sentence_lengths)
# print(max_length)

31


In [24]:
# padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [26]:
# embedding_matrix = np.zeros((vocab_size, 300))

In [28]:
# nlp = spacy.load('en_vectors_web_lg')

In [29]:
# for word, i in t.word_index.items():
#     embedding_vector = nlp.vocab.get_vector(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

In [85]:
# define model
model = Sequential()
# e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=31, trainable=False)
# model.add(e)
model.add(Dense(50, input_shape=(300,)))
model.add(Dense(3, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 50)                15050     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 153       
Total params: 15,203
Trainable params: 15,203
Non-trainable params: 0
_________________________________________________________________
None


In [86]:
renum_labels = []
for label in labels:
    if label == 0:
        renum_labels.append(0)
    elif label == 2:
        renum_labels.append(1)
    else:
        renum_labels.append(2)

In [81]:
categorical_labels = to_categorical(renum_labels, num_classes=3)

In [90]:
from sklearn.model_selection import train_test_split

In [94]:
X_train, X_test, y_train, y_test = train_test_split(matrix_embedding,
categorical_labels,
test_size=0.2,
random_state=40)

In [95]:
model.fit(X_train, y_train, epochs=5, verbose=0)

In [98]:
# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 77.000000
